# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding = "utf-8" )
weather_data = weather_data.rename(columns = {"Longtidue":"Longitude"})
weather_data.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Touros,100,BR,1584146424,100,-5.20,-35.46,75.20,10.60
1,Port Alfred,26,ZA,1584146424,97,-33.59,26.89,69.01,4.00
2,Dabakala,26,CI,1584146424,67,8.37,-4.43,81.41,5.93
3,Jumla,23,NP,1584146424,65,29.27,82.18,25.75,7.40
4,Kapaa,90,US,1584146143,64,22.08,-159.32,77.00,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [25]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 90)]
ideal_weather = ideal_weather.loc[(weather_data['Max Temp'] > 70)]
ideal_weather = ideal_weather.loc[(weather_data['Wind Speed'] < 20)]
ideal_weather = ideal_weather.loc[weather_data['Cloudiness'] < 25 ]
 
hotel_df = ideal_weather.dropna().reset_index(drop=True)
print(len(hotel_df))

60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Cidreira,0,BR,1584146424,90,-30.18,-50.21,76.08,11.65,
1,Hilton Head,1,US,1584146370,77,32.22,-80.75,73.99,5.82,
2,Kavieng,12,PG,1584146194,77,-2.57,150.80,84.24,12.10,
3,Mar del Plata,0,AR,1584146181,68,-38.00,-57.56,77.00,10.29,
4,Lázaro Cárdenas,0,MX,1584146429,71,17.96,-102.20,79.93,7.23,


In [38]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for x in hotel_df['Latitude']:
    params = {
        "location" : f"{x},{hotel_df['Longitude']}",
        "radius": 5000,
        "key": g_key,
        "keyword" : "hotel"
    }
    
    response = requests.get(url,params).json()
    hotel_df["Hotel Name"].append(response['results'][0]['name'])
    
hotel_df

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map